# II. Data_Unification

Índice

### Import Libraries and PyPower.py 

In [1]:
import pandas as pd
from sys import displayhook
import sys
sys.path.append('../Python')
from  PyPower import *

### Otter DataFrame

In [2]:
otter = pd.read_csv('../../data/1-clean/otter_clean.csv')
otter_original_shape= otter.shape
otter.head()

,fecha,marca,local,canal,id_orden,estado_orden,subtotal,impuesto,descuento
0,2023-11-03 15:55:55,Umbrella SH,EMEA - SP - MAD - Tetuan,justeat,138656463,Nuevo,10,0,0
1,2023-11-03 15:49:40,Umbrella SH,EMEA - SP - MAD - Tetuan,justeat,138656912,Nuevo,"22,4",0,0
2,2023-11-03 15:49:02,Umbrella SH,EMEA - SP - MAD - Tetuan,glovo,MSDGW9UM,Entregado,"10,4",0,0
3,2023-11-03 15:48:41,Umbrella SH,Umbrella SH - Calle Linares,ubereats,8D40B,Nuevo,"36,6",0,"-13,9"
4,2023-11-03 15:44:25,Umbrella SH,Goleta,ubereats,DF131,Entregado,"31,8",0,"-13,9"


In [3]:
otter.describe(include='all')

,fecha,marca,local,canal,id_orden,estado_orden,subtotal,impuesto,descuento
count,534,528,534,534,534,534,534,534,534
unique,530,3,8,4,534,3,253,18,13
top,2023-11-01 20:23:57,Umbrella SH,EMEA - SP - MAD - Tetuan,ubereats,138656463,Entregado,"31,8",0,0
freq,2,443,132,353,1,464,35,517,315


#### Columns marca and local (delete rows)
For the scope of the project, we will select only the brands "Tepuy Burger" and "Tepuy Basics", and localization in "Tepuy" only

In [4]:
otter.marca.value_counts()

marca
Umbrella SH     443
Tepuy Burger     60
Tepuy Basics     25
Name: count, dtype: int64

In [5]:
filtered_df = otter[otter['marca'].isin(['Tepuy Burger', 'Tepuy Basics'])]

filtered_df.marca.value_counts()

marca
Tepuy Burger    60
Tepuy Basics    25
Name: count, dtype: int64

In [6]:
otter = filtered_df.copy()

In [7]:
otter.local.value_counts()

local
Tepuy Burger - Rodolfo Salazar          29
Tepuy Burger - Benidorm                 22
Tepuy Burger - Calle Bazan               9
Umbrella SH - San Vicent del Raspeig     9
Goleta                                   8
EMEA - SP - MAD - Salamanca              5
Umbrella SH - Calle Linares              3
Name: count, dtype: int64

In [8]:
filtered_df = otter[otter['local'].isin(['Tepuy Burger - Rodolfo Salazar', 'Tepuy Burger - Benidorm', 'Tepuy Burger - Calle Bazan'])]

filtered_df.local.value_counts()

local
Tepuy Burger - Rodolfo Salazar    29
Tepuy Burger - Benidorm           22
Tepuy Burger - Calle Bazan         9
Name: count, dtype: int64

In [9]:
otter = filtered_df.copy()

#### Columns Float Subtotal

In [10]:
#The data is going a SQL database, so we need the numbers with . instead of ,
otter.subtotal = otter.subtotal.apply(replace_to, change_for=',', change_to='.')
otter.impuesto = otter.impuesto.apply(replace_to, change_for=',', change_to='.')
otter.descuento = otter.descuento.apply(replace_to, change_for=',', change_to='.')

In [11]:
otter.describe()

,fecha,marca,local,canal,id_orden,estado_orden,subtotal,impuesto,descuento
count,60,60,60,60,60,60,60,60,60
unique,58,1,3,3,60,3,50,2,3
top,2023-11-01 20:14:55,Tepuy Burger,Tepuy Burger - Rodolfo Salazar,glovo,AULRBGNM,Entregado,26.9,0,0
freq,2,60,29,41,1,54,3,59,58


#### Rename columns to english names

In [12]:
columns_to_rename={'fecha':'created_datetime',
                   'marca':'brand',
                   'local':'location',
                   'canal':'channel',
                   'id_orden':'id_order',
                   'impuesto':'vat',
                   'descuento':'discount',
                   'estado_orden':'status'}

otter = rename_columns(otter, columns_to_rename)


In [13]:
otter.head()

,created_datetime,brand,location,channel,id_order,status,subtotal,vat,discount
16,2023-11-03 14:46:46,Tepuy Burger,Tepuy Burger - Rodolfo Salazar,glovo,AULRBGNM,Entregado,26.9,0,0
18,2023-11-03 14:40:15,Tepuy Burger,Tepuy Burger - Benidorm,glovo,B1YBHZJW,Entregado,51.5,0,0
22,2023-11-03 14:23:06,Tepuy Burger,Tepuy Burger - Rodolfo Salazar,glovo,ALMVLKTB,Entregado,26.9,0,0
50,2023-11-02 22:32:27,Tepuy Burger,Tepuy Burger - Benidorm,glovo,BXVJAQHE,Entregado,40.9,0,0
53,2023-11-02 22:28:20,Tepuy Burger,Tepuy Burger - Calle Bazan,glovo,ABUGGFKZ,Cancelado,20.4,0,0


In [14]:
otter.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60 entries, 16 to 509
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   created_datetime  60 non-null     object
 1   brand             60 non-null     object
 2   location          60 non-null     object
 3   channel           60 non-null     object
 4   id_order          60 non-null     object
 5   status            60 non-null     object
 6   subtotal          60 non-null     object
 7   vat               60 non-null     object
 8   discount          60 non-null     object
dtypes: object(9)
memory usage: 4.7+ KB


#### Columns subtotal, vat and discount are object type and they should be float

In [15]:
#PyPower method
otter= change_type(otter, 'subtotal', 'object','float')
otter= change_type(otter, 'vat', 'object','float')
otter= change_type(otter, 'discount', 'object','float') 

### Deliverect DataFrame

In [16]:
deliverect = pd.read_csv('../../data/1-clean/deliverect_clean.csv')
deliverect_original_shape= otter.shape
deliverect.head()

,createdtimeutc,location,orderid,channel,status,type,payment,paymentamount,deliverycost,discounttotal,tip,drivertip,subtotal,vat
0,2023-10-31 21:31:15,Alicante - Bazan,138509659,Just Eat,ACCEPTED,DELIVERY,CARD,85.39,1.99,0.0,0.0,0.0,83.4,7.57
1,2023-10-31 21:02:26,Alicante - Periodista Rodolfo Salazar,138502564,Just Eat,DELIVERED,DELIVERY,CARD,41.89,1.99,0.0,0.0,0.0,39.9,3.62
2,2023-10-29 20:20:15,Alicante - Periodista Rodolfo Salazar,138409768,Just Eat,DELIVERED,DELIVERY,CARD,31.09,1.99,0.0,0.0,0.0,29.1,2.65
3,2023-10-29 19:53:10,Alicante - Periodista Rodolfo Salazar,138402233,Just Eat,DELIVERED,DELIVERY,CARD,19.69,1.99,0.0,0.0,0.0,17.7,1.61
4,2023-10-29 19:28:17,Alicante - Periodista Rodolfo Salazar,138390234,Just Eat,DELIVERED,DELIVERY,CARD,15.99,2.99,0.0,0.0,0.0,13.0,1.19


In [17]:
deliverect.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   createdtimeutc  62 non-null     object 
 1   location        62 non-null     object 
 2   orderid         62 non-null     int64  
 3   channel         62 non-null     object 
 4   status          62 non-null     object 
 5   type            62 non-null     object 
 6   payment         62 non-null     object 
 7   paymentamount   62 non-null     float64
 8   deliverycost    62 non-null     float64
 9   discounttotal   62 non-null     float64
 10  tip             62 non-null     float64
 11  drivertip       62 non-null     float64
 12  subtotal        62 non-null     float64
 13  vat             62 non-null     float64
dtypes: float64(7), int64(1), object(6)
memory usage: 6.9+ KB


##### For the project, the customer only wants to unify the data from the delivery platforms
Delete where type is not DELIVERY

In [18]:
filtered_df = deliverect[deliverect['type']=='DELIVERY']

deliverect = filtered_df.copy()

##### Unify columns names

In [19]:
columns_to_rename={'createdtimeutc':'created_datetime',
                   'payment':'payment_method',
                   'paymentamount':'total',
                   'deliverycost':'delivery_cost',
                   'orderid':'id_order',
                   'discounttotal':'discount'}

deliverect = rename_columns(deliverect, columns_to_rename)


In [20]:
deliverect.head()

,created_datetime,location,id_order,channel,status,type,payment_method,total,delivery_cost,discount,tip,drivertip,subtotal,vat
0,2023-10-31 21:31:15,Alicante - Bazan,138509659,Just Eat,ACCEPTED,DELIVERY,CARD,85.39,1.99,0.0,0.0,0.0,83.4,7.57
1,2023-10-31 21:02:26,Alicante - Periodista Rodolfo Salazar,138502564,Just Eat,DELIVERED,DELIVERY,CARD,41.89,1.99,0.0,0.0,0.0,39.9,3.62
2,2023-10-29 20:20:15,Alicante - Periodista Rodolfo Salazar,138409768,Just Eat,DELIVERED,DELIVERY,CARD,31.09,1.99,0.0,0.0,0.0,29.1,2.65
3,2023-10-29 19:53:10,Alicante - Periodista Rodolfo Salazar,138402233,Just Eat,DELIVERED,DELIVERY,CARD,19.69,1.99,0.0,0.0,0.0,17.7,1.61
4,2023-10-29 19:28:17,Alicante - Periodista Rodolfo Salazar,138390234,Just Eat,DELIVERED,DELIVERY,CARD,15.99,2.99,0.0,0.0,0.0,13.0,1.19


In [24]:
deliverect.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61 entries, 0 to 61
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   created_datetime  61 non-null     object 
 1   location          61 non-null     object 
 2   id_order          61 non-null     object 
 3   channel           61 non-null     object 
 4   status            61 non-null     object 
 5   type              61 non-null     object 
 6   payment_method    61 non-null     object 
 7   total             61 non-null     float64
 8   delivery_cost     61 non-null     float64
 9   discount          61 non-null     float64
 10  tip               61 non-null     float64
 11  drivertip         61 non-null     float64
 12  subtotal          61 non-null     float64
 13  vat               61 non-null     float64
dtypes: float64(7), object(7)
memory usage: 7.1+ KB


In [22]:
deliverect = change_type(deliverect, 'id_order', 'int64','object')

### Export documents

In [25]:
otter.to_csv('../../data/2-processed/otter_processed.csv', index=False)
deliverect.to_csv('../../data/2-processed/deliverect_processed.csv', index=False)